# Feature Selection and Feature Engineering

We want to do our best to make good predictions by creating good models. One way we can improve our model is to consider the data's feature and either specifically _select_ features (**feature selection**) and/or _create new features_ (called **feature engineering**)

# Learning Objectives

- Use correlations and other algorithms to inform feature selection
- Address the problem of multicollinearity in regression problems
- Create new features for use in modeling
    - Use `PolynomialFeatures` to build compound features

In [2]:
# Initial imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Feature Selection

New dataset for today! Insurance costs

My source: https://www.kaggle.com/mirichoi0218/insurance (they got the idea for cleaning up the original open source data from [Machine Learning with R](https://www.packtpub.com/product/machine-learning-with-r-third-edition/9781788295864))

Target: `charges`

In [3]:
# Read in the data
df = pd.read_csv('data/insurance.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/insurance.csv'

In [ ]:
# explore the data
df.info()

In [ ]:
# visualize relationships between numeric columns
sns.pairplot(df);

In [ ]:
# visualize correlations between numeric columns
sns.heatmap(df.corr(), annot=True);

#### Observations?

- 


In [ ]:
df.describe(include=['O'])

### First Things First - Train Test Split!

In [ ]:
# Import train test split
from sklearn.model_selection import train_test_split

In [ ]:
# Define our X and y
X = df.drop(columns='charges')
y = df['charges']

In [ ]:
# Train test split
# Use test_size=0.25, random_state=42
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

In [ ]:
X_train.describe(include=['O'])

In [ ]:
X_test.describe(include=['O'])

### Need to Encode!

In [ ]:
# Import our One Hot Encoder
from sklearn.preprocessing import OneHotEncoder
# Going to use ColumnTransformer to encode only cat cols
from sklearn.compose import ColumnTransformer

In [ ]:
# Which columns are categoricals?
# Handy trick!
cat_cols = [c for c in df.columns if df[c].dtype == 'O']
cat_cols

In [ ]:
# create an encoder object. This will help us to convert
# categorical variables to new columns
encoder = OneHotEncoder(handle_unknown='error',
                        drop='first', # leaves all 4 regions, creating 4 new columns for region
                        categories='auto')

# Create an columntransformer object.
# This will help us to merge transformed columns
# with the rest of the dataset.

ct = ColumnTransformer(transformers=[('ohe', encoder, cat_cols)],
                       remainder='passthrough')
ct.fit(X_train)
X_train_enc = ct.transform(X_train)
X_test_enc = ct.transform(X_test)

In [ ]:
X_train_enc

In [ ]:
# can display as a dataframe like so
X_train_enc = pd.DataFrame(X_train_enc, columns= ct.get_feature_names())
X_train_enc.head(10)

### Now To Scale!

In [ ]:
# Import our scaler
from sklearn.preprocessing import StandardScaler
# intstantiate our scaler
scaler = StandardScaler()

# train on train data
scaler.fit(X_train_enc)

# transform both train and test data
X_train_scaled = scaler.transform(X_train_enc)
X_test_scaled = scaler.transform(X_test_enc)

In [ ]:
pd.DataFrame(X_train_scaled, columns= ct.get_feature_names())

### And let's model!

In [ ]:
# Import our linear regression function
from sklearn.linear_model import LinearRegression
# instantiate
lr = LinearRegression()

# fit
lr.fit(X_train_scaled, y_train)

# grab predictions for train and test set
train_preds = lr.predict(X_train_scaled)
test_preds = lr.predict(X_test_scaled)

In [ ]:
lr.coef_

In [ ]:
# evaluate
from sklearn.metrics import r2_score, mean_squared_error

r2_score(y_train, train_preds)

In [ ]:
r2_score(y_test, test_preds)

In [ ]:
# !conda install -c districtdatalabs yellowbrick

In [ ]:
# visualizing our residuals 
# https://www.scikit-yb.org/en/latest/api/regressor/residuals.html
from yellowbrick.regressor import ResidualsPlot

visualizer = ResidualsPlot(lr)

visualizer.fit(X_train_scaled, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test_scaled, y_test)  # Evaluate the model on the test data
visualizer.show()  
plt.show()

In [ ]:
# Basically the same as
plt.scatter(train_preds, (train_preds-y_train))
plt.scatter(test_preds, (test_preds-y_test), color='green');

Ideas to continue improving this model?

- 


## Feature Importance through Coefficients

Because we've scaled our data, we can explore our coefficients to see which are having more of an impact on our model.

Note! This, or using p-values from a statsmodels model, is all you're expected to do in this project!

In [ ]:
# Let's make our scaled training data a df, for ease of use
X_train_scaled = pd.DataFrame(X_train_scaled, columns=ct.get_feature_names())
X_train_scaled.head()

In [ ]:
# Same with X_test_scaled
X_test_scaled = pd.DataFrame(X_test_scaled, columns=ct.get_feature_names())

In [ ]:
# Check the coefficients
lr.coef_

In [ ]:
# look at the coefficients with the names of each col
dict(zip(X_train_scaled.columns, lr.coef_))

In [ ]:
# sns.heatmap(X_train_scaled.corr(), annot=True);

In [ ]:
# Let's model again using only the 3 strongest coefficients
used_cols = ['ohe__x2_southeast', 'ohe__x1_yes', 'age']

In [ ]:
lr_subset = LinearRegression()

lr_subset.fit(X_train_scaled[used_cols], y_train)

In [ ]:
lr_subset.score(X_train_scaled[used_cols], y_train)

In [ ]:
lr_subset.score(X_test_scaled[used_cols], y_test)

In [ ]:
# visualizing our residuals
# https://www.scikit-yb.org/en/latest/api/regressor/residuals.html

visualizer = ResidualsPlot(lr_subset)

visualizer.fit(X_train_scaled[used_cols], y_train)  # Fit the training data to the visualizer
visualizer.score(X_test_scaled[used_cols], y_test)  # Evaluate the model on the test data
visualizer.show()  
plt.show()

#### Evaluate

- 


# Feature Engineering

## Interaction Terms

When do we need interaction terms? And how do we check for them?

Well, first things first - what interactions do _you_ think would make sense? That's the easiest way to incorporate interaction terms - use domain knowledge to think through what usefully could be combined into an interaction.

As for how to check if something might be better captured as an interaction...

In [ ]:
X_train_enc

In [ ]:
# add the target back onto our OHE df
# note the index difference...
train_df = X_train_enc.copy()
train_df['target'] = y_train.reset_index(drop=True)
train_df

In [ ]:
sns.lmplot(x='age', y='target', data=train_df, scatter=False)

In [ ]:
# an example of no interaction term...
sns.lmplot(x='age', y='target', hue='ohe__x1_yes', data=train_df, scatter=False)
plt.show()

How do I know these two variables, `age` and `smoker_yes`, aren't interacting? 

- Look at the slopes - parallel


In [ ]:
sns.lmplot(x='bmi', y='target', data=train_df, scatter=False);

In [ ]:
# now let's look at something else...
sns.lmplot(x='bmi', y='target', hue='ohe__x1_yes', data=train_df, scatter=False)
plt.show()

What do you think?

- 


## Polynomial Features

Instead of just multiplying features at random, we might consider trying **every possible product of features**. That's what PolynomialFeatures does.


https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

Demonstrating this on a toy example, with a single x variable predicting y.

In [ ]:
# 150 samples from uniform distribution between -2pi and 2pi

x = np.random.uniform(-2*np.pi, 2*np.pi, 150)

# Creating target (y) - so we know the true relationship between x and y
# But - adding some noise (error) with 'np.random'

y = np.sin(x) + np.random.normal(loc=0, scale=0.4, size=len(x))

In [ ]:
# Visualize it
plt.scatter(x, y)

plt.ylabel('$\sin(x)$ plus noise')
plt.xlabel('x values are randomly chosen from $[-2\pi, 2\pi]$')
plt.show()

In [ ]:
# Fitting a linear model
lr = LinearRegression()
lr.fit(x.reshape(-1, 1), y)

In [ ]:
# Grabbing the predicted values
y_pred = lr.predict(x.reshape(-1, 1))

In [ ]:
# Scoring our model
lr.score(x.reshape(-1, 1), y)

In [ ]:
# Visualize it
plt.scatter(x, y) # original data

plt.plot(x, y_pred, c='red') # predicted values

plt.ylabel('$\sin(x)$ + noise')
plt.xlabel('x values randomly chosen between $-2\pi$ and $2\pi$')
plt.title("Simple Linear Regression")

plt.show()

Is this a good model? Well - of course not. It's definitely **underfit** - it is not complex enough to accurately capture the pattern and predict the target.

Let's try again, but now with polynomials!

In [ ]:
# Import polynomial features
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# For this, we'll need some helper functions
# Shoutout to Andy for sending me these

def create_poly_dataset(x, degree):
    """
    returning dataset with the given polynomial degree
    """
    # Instantiate the PolynomialFeatures object with given 'degree'
    poly = PolynomialFeatures(degree=degree)

    # Now transform data to create higher order features
    new_data = poly.fit_transform(x.reshape(-1, 1))
    return new_data

def fit_linear_model(data, y):
    """
    fitting a linear model and printing model details
    """
    np.set_printoptions(precision=4, suppress=True)

    if data.ndim == 1:
        data = data.reshape(-1, 1)

    lr = LinearRegression(fit_intercept=False)
    lr.fit(data, y)
    print("-"*13)
    print("Coefficients: ", lr.coef_)
    y_pred = lr.predict(data)
    print(f"R-Squared: {lr.score(data, y):.3f}")
    return lr

def plot_predict(x, y, model):
    """
    plotting predictions against true values
    """
    plt.scatter(x, y, label='true')
    x_pred = np.linspace(x.min(), x.max(), 100)
    
    # visualize beyond this x range by uncommenting below:
#     extra = x.ptp() * .2
#     x_pred = np.linspace(x.min() - extra, x.max() + extra, 100)

    plt.plot(x_pred, model.predict(create_poly_dataset(x_pred, len(model.coef_)-1)),
             label='predicted', c='red')

    if len(model.coef_) == 1:
        plt.title(f"{len(model.coef_) - 1} Polynomial Terms \n (no slope)")
    elif (len(model.coef_) - 1) == 1:
        plt.title(f"{len(model.coef_) - 1} Polynomial Term")
    else:
        plt.title(f"{len(model.coef_) - 1} Polynomial Terms")

    plt.legend()
    plt.show()
    return

In [ ]:
# visualizing an assortment of polynomial degrees
# can visualize each sequential polynomial with `range(n)`
for i in [0, 1, 2, 3, 5, 7, 9, 13, 18]:
    xi = create_poly_dataset(x, i)
    plot_predict(x, y, fit_linear_model(xi, y))

Evaluate: which of these is the best?

- 


Evaluate: so what?

- 


### Now on our dataset

In [ ]:
# First all polynomial terms - to degree 3
poly = PolynomialFeatures(degree=3)

In [ ]:
# fit that preprocessor
poly.fit(X_train_scaled)

In [ ]:
# transform our training and testing data
X_train_deg3 = poly.transform(X_train_scaled)
X_test_deg3 = poly.transform(X_test_scaled)

In [ ]:
# Let's explore our new inputs
X_train_deg3.shape

In [ ]:
# Make it into a dataframe to more easily see what's happening
deg_3_df = pd.DataFrame(X_train_deg3, 
                        # column names come from the combinations
                        columns=poly.get_feature_names(input_features=X_train_scaled.columns))
# Check out the resulting columns
deg_3_df.columns

In [ ]:
# Interaction-only terms with the same function
# Set interaction_only=True
interactions = PolynomialFeatures(degree=2, interaction_only=True)

In [ ]:
# Fit on training data
interactions.fit(X_train_scaled)

# Transform training and testing data
X_train_interactions = interactions.transform(X_train_scaled)
X_test_interactions = interactions.transform(X_test_scaled)

In [ ]:
X_train_interactions.shape

In [ ]:
# Can make into a df to explore
int_df = pd.DataFrame(X_train_interactions,
                      columns=interactions.get_feature_names(input_features=X_train_scaled.columns))
int_df.columns

Evaluate: What do you think? Is this blanket way of approaching polynomial or interaction terms useful?

- probably not unless you use something like Recursive Feature Elimination!
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html?highlight=rfe#sklearn.feature_selection.RFE


In [ ]:
# If we want to then eliminate features...
from sklearn.feature_selection import RFE

# Make a new model to explore how each feature impacts the model
lr_rfe = LinearRegression()

# Instantiate our RFE with that new model and how many features we want
rfe = RFE(lr_rfe, n_features_to_select=3)

# Fit to our polynomial training data
rfe.fit(X_train_deg3, y_train)

In [ ]:
# Ranking of the columns in terms of usefulness to that LinearRegression model
rfe.ranking_

In [ ]:
# Explore which columns are the top 3 
# If it's a chosen feature, then .support_ will be True
dict(zip(deg_3_df.columns, rfe.support_))

In [ ]:
# Here are the three columns it found most important from that
for col_name, support in dict(zip(deg_3_df.columns, rfe.support_)).items():
    if support == True:
        print(col_name)

## Feature Selection and Feature Importances...

Not much time to do this, but:

- Lasso Regression (L1 regularization)
- Recursive Feature Elimination
- Forward Stepwise Selection

Can always check out the python library [`eli5`](https://eli5.readthedocs.io/en/latest/index.html) (yes, Explain Like I'm 5)

## Resources:

[Feature Engineering and Selection: A Practical Approach for Predictive Models](https://bookdown.org/max/FES/) (computing done in R, but book focuses mostly on discussing the hows and whys rather than focusing on implementation)

- Their chapter on [Encoding Categorical Predictors](https://bookdown.org/max/FES/encoding-categorical-predictors.html)
- And their chapter on [Detecting Interaction Effects](https://bookdown.org/max/FES/detecting-interaction-effects.html)